## CloudNativePG (CNPG)

![](https://cloudnative-pg.io/documentation/current/images/k8s-pg-architecture.png)

Quelle: https://cloudnative-pg.io/documentation/current/architecture/

- - -

**CloudNativePG** ist ein Kubernetes-Operator, der das Management von **PostgreSQL-Datenbanken** in einem Kubernetes-Cluster automatisiert. Er ermöglicht **hochverfügbare, skalierbare und containerisierte PostgreSQL-Cluster**.

### Architektur

Dieser Cluster besteht aus:

- **CNPG Operator** → Verwaltet den PostgreSQL-Cluster in Kubernetes.
- **Datenbank-Cluster (`database-cluster`)**
  - **Primary-Instanz (`database-cluster-1`)** → Die Haupt-Datenbankinstanz, auf die geschrieben wird.
  - **Replica-Instanzen (`database-cluster-2`, `database-cluster-3`)** → Replikationen für Skalierung und Hochverfügbarkeit.
- **Kubernetes Services**
  - **`database-cluster-rw` (Read/Write Service)** → Für den Zugriff auf die **Primary-Instanz**.
  - **`database-cluster-ro` (Read-Only Service)** → Für **nur Lesezugriffe** auf die Replicas.

- - -

Installation mittels `helm`. Alle anderen Varianten waren nicht erfolgreich.

In [ ]:
%%bash
helm repo add cnpg https://cloudnative-pg.github.io/charts
helm upgrade --install cnpg \
  --namespace cnpg-system \
  --create-namespace \
  cnpg/cloudnative-pg

Helm chart to install a CloudNativePG database cluster.

In [ ]:
%%bash
helm repo add cnpg https://cloudnative-pg.github.io/charts
helm upgrade --install database \
  --namespace database \
  --create-namespace \
  cnpg/cluster

Zusätzlich installieren wir das PostgreSQL CLI

In [ ]:
%%bash
sudo apt-get update && sudo apt-get install postgresql-client -y

Wie üblich öffnen wir den Port, damit wir einfacher auf die Datenbank zugreifen können:

In [ ]:
%%bash
kubectl patch svc database-cluster-rw -n database -p '{"spec":{"type":"NodePort"}}'
echo "postgresql://"$(cat ~/work/server-ip)":$(kubectl get svc database-cluster-rw -n database -o=jsonpath='{.spec.ports[0].nodePort}')/"


Jetzt gibt es verschiedene Möglichkeiten sich mit postgreSQL zu verbinden:
    
Ohne Password mit pgsql CLI im Container

    kubectl exec -it database-cluster-1 -n database -- psql -U postgres
    
Mit Password     

In [ ]:
%%bash
echo "ssh -i ~/.ssh/lerncloud ubuntu@$(cat ~/work/server-ip)"
echo psql -h $(cat ~/work/server-ip) -p $(kubectl get svc database-cluster-rw -n database -o=jsonpath='{.spec.ports[0].nodePort}') -U postgres

### Part 4: Monitor clusters with Prometheus and Grafana

* [Prometheus and Grafana](https://github.com/cloudnative-pg/cloudnative-pg/blob/main/docs/src/quickstart.md#part-4-monitor-clusters-with-prometheus-and-grafana)
* [Bootstrap](https://github.com/cloudnative-pg/cloudnative-pg/blob/main/docs/src/bootstrap.md)